In [1]:
import warnings
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_theme()

# Read and explore the data

In [2]:
df = pd.read_csv('../data/raw/train.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1166915 entries, 0 to 1166914
Data columns (total 10 columns):
 #   Column              Non-Null Count    Dtype  
---  ------              --------------    -----  
 0   id                  1166915 non-null  int64  
 1   vendor_id           1166915 non-null  int64  
 2   pickup_datetime     1166915 non-null  object 
 3   passenger_count     1166915 non-null  int64  
 4   pickup_longitude    1166915 non-null  float64
 5   pickup_latitude     1166915 non-null  float64
 6   dropoff_longitude   1166915 non-null  float64
 7   dropoff_latitude    1166915 non-null  float64
 8   store_and_fwd_flag  1166915 non-null  object 
 9   trip_duration       1166915 non-null  int64  
dtypes: float64(4), int64(4), object(2)
memory usage: 89.0+ MB


In [3]:
df.describe().round()

,id,vendor_id,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,trip_duration
count,1166915.0,1166915.0,1166915.0,1166915.0,1166915.0,1166915.0,1166915.0,1166915.0
mean,583457.0,2.0,2.0,-74.0,41.0,-74.0,41.0,959.0
std,336859.0,0.0,1.0,0.0,0.0,0.0,0.0,5625.0
min,0.0,1.0,0.0,-122.0,34.0,-122.0,32.0,1.0
25%,291728.0,1.0,1.0,-74.0,41.0,-74.0,41.0,397.0
50%,583457.0,2.0,1.0,-74.0,41.0,-74.0,41.0,662.0
75%,875186.0,2.0,2.0,-74.0,41.0,-74.0,41.0,1075.0
max,1166914.0,2.0,9.0,-61.0,52.0,-61.0,44.0,3526282.0
